In [ ]:
# get all season 13reports. landing page https://sites.google.com/a/marssociety.org/mdrs2013/crew-reports
from bs4 import BeautifulSoup as bs
import requests
import urllib.request
import csv
import pandas as pd
import re

In [ ]:
# get links to individual reports from the landing page.
url = urllib.request.urlopen('https://sites.google.com/a/marssociety.org/mdrs2013/previous-field-seasons/2013-field-season/crew-reports').read()
soup = bs(url, 'lxml')

In [ ]:
links = [] #there are links to same reports; will have to remove duplicates from final dateframe 

for x in soup.find_all('tbody'):
    for y in x.find_all('a'):
        if 'crew-' in str(y['href']) in str(y['href']):              
            links.append(y['href'])    
            
links = list(set(links)) #remove duplicates   
#print(links)






In [ ]:
broken_link = 'http://mdrs.marssociety.org/home/crew-140/422-commanderreport'

#links that do not have sites.google.com/a/ do not work. Need to add that part to links
#http://mdrs.marssociety.org/home/crew-140/422-commanderreport
#https://sites.google.com/a/marssociety.org/mdrs2013/home/crew-133/112-greenhabreport
#https://sites.google.com/a/mdrs.marssociety.org/mdrs2013/home/crew-140/422-commanderreport
#https://sites.google.com/a/mdrs.marssociety.org/home/crew-140/422-commanderreport

#'https://sites.google.com/a/'+ broken_link.split('//')[1]


#'mdrs.marssociety.org/home/crew-140/422-commanderreport'


#broken_link.rsplit('/',2)[1] + '/' + broken_link.rsplit('/', 2)[2]
#broken_link.split('//')[1].split('.',1)[1]


In [ ]:
# process single link
#link = 'https://sites.google.com/a/marssociety.org/mdrs2013/home/crew-139/411-commandercheckinreport'

#url1 = urllib.request.urlopen(link).read()
#link_soup = bs(url1, 'lxml')


#check for good links - when bad link, put text as None 
# to fix broken links
#'https://sites.google.com/a/' + broken_link.split('//')[1].split('.',1)[1].split('/',3)[0] + '/mdrs2013/home/' + broken_link.rsplit('/',2)[1] + '/' + broken_link.rsplit('/', 2)[2] #'mdrs.marssociety.org/mdrs2013/home/'

fixed_links = []

for link in links:
    try:
        url1 = urllib.request.urlopen(link).read()
        fixed_links.append(link)
    except urllib.error.HTTPError as exception:
        link1 = 'https://sites.google.com/a/' + link.split('//')[1].split('.',1)[1].split('/',3)[0] + '/mdrs2013/home/' + link.rsplit('/',2)[1] + '/' + link.rsplit('/', 2)[2] 
        try:
            url2 = urllib.request.urlopen(link1).read()
            fixed_links.append(link1)
        except urllib.error.HTTPError as exception:
            print (link)
            print (link1)
            print (exception)
                
        






In [ ]:
link_soup.h3.get_text().lower() # title 

In [ ]:
#link.rsplit('/',2)[-2] #crew
#link.rsplit('/',1)[-1].split('-')[0] #date

In [ ]:
#link_soup.find('tbody') #html
link_soup.find('tbody').get_text() #text

In [ ]:
#process all ;links
#to check whether report is a good catch-all for reports
# missing the following reports:
# https://sites.google.com/a/marssociety.org/mdrs2013/home/crew-136/218-commandercheckin
#https://sites.google.com/a/marssociety.org/mdrs2013/home/crew-136/217-commandercheckin
crew = []
date = []
title =[]
text = []
html = [] 
url_links = []



for l in fixed_links:
    url1 = urllib.request.urlopen(l).read()
    link_soup = bs(url1, 'lxml')
    if 'report' in link_soup.h3.get_text().lower():
        print(link_soup.h3.get_text().lower())
        url_links.append(l) #link
        title.append(link_soup.h3.get_text().lower()) #title
        crew.append(l.rsplit('/',2)[-2]) # crew number 
        date.append(l.rsplit('/',1)[-1].split('-')[0]) #date
        html.append(link_soup.tbody) #html
        text.append(link_soup.tbody.get_text()) # text
        

        



In [ ]:
reports = pd.DataFrame({'crew': crew, 'date': date, 'header': title, 'body':text, 'html': html, 'link': url_links})


#fix crew-xx into xx
for i in range(len(reports)):
    reports['crew'].iloc[i] = reports['crew'].iloc[i].split("crew-")[1]
   # print(i, reports['crew'].iloc[i])

reports.to_csv('s13_reports.csv')
